In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import json
import glob
import ast
from tqdm.auto import tqdm
import numpy as np


state_code = pd.read_csv('/content/gdrive/MyDrive/Colab/COMS6998EMODS/state_code.csv')
states_dict = {}
for index, row in state_code.iterrows():
  states_dict[row['name'].lower()] = row['code']
states_dict

In [124]:


HOME_DIR = '/content/gdrive/MyDrive/Colab/COMS6998EMODS/CovidData/States_Data/*'
pfile = glob.glob(HOME_DIR)

# pfile = ['/content/gdrive/MyDrive/Colab/COMS6998EMODS/CovidData/national-history.csv']
# pfile = pfile[:1]
for path in tqdm(pfile):
  fname = path.split("/")[-1].split("-history")[0].replace("-", " ")
  if fname == 'us virgin islands':
    fname = 'virgin islands'
  code = states_dict[fname]

  df = pd.read_csv(path, engine='python')
  df['date'] = pd.to_datetime(df['date'])
  mask = (df['date'] >= '2020-08-01') & (df['date'] <= '2020-12-31')
  df = df.loc[mask]
  df = df.sort_values(by='date')

  # df.to_csv("/content/gdrive/MyDrive/Colab/COMS6998EMODS/CovidData/notional_all.csv", index=False)

  new_df = df[['date', 'deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 'positiveIncrease',  'totalTestResultsIncrease']]
  new_df = new_df.reset_index(drop=True)

  new_df['deathIncreaseRate']=new_df['deathIncrease'].abs().pct_change()
  new_df['hospitalizedIncreaseRate']=new_df['hospitalizedIncrease'].abs().pct_change() 
  new_df['negativeIncreaseRate']=new_df['negativeIncrease'].abs().pct_change() 
  new_df['positiveIncreaseRate']=new_df['positiveIncrease'].abs().pct_change() 
  new_df['totalTestResultsIncreaseRate']=new_df['totalTestResultsIncrease'].abs().pct_change()

  # new_df.replace([np.inf, -np.inf], 1, inplace=True)
  new_df.replace(np.nan, 0, inplace=True)

  new_df['deathIncreaseRate'] = new_df['deathIncreaseRate'] * 100
  new_df['hospitalizedIncreaseRate'] = new_df['hospitalizedIncreaseRate'] * 100
  new_df['negativeIncreaseRate'] = new_df['negativeIncreaseRate'] * 100
  new_df['positiveIncreaseRate'] = new_df['positiveIncreaseRate'] * 100
  new_df['totalTestResultsIncreaseRate'] = new_df['totalTestResultsIncreaseRate'] * 100

  new_df.to_csv("/content/gdrive/MyDrive/Colab/COMS6998EMODS/CovidData/States_Data_Rate/" + code+".csv", index=False)
new_df

  0%|          | 0/56 [00:00<?, ?it/s]

,date,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,deathIncreaseRate,hospitalizedIncreaseRate,negativeIncreaseRate,positiveIncreaseRate,totalTestResultsIncreaseRate
0,2020-08-01,1,2,0,144,5527,0.0,0.000000,0.0,0.000000,0.000000
1,2020-08-02,0,1,0,136,4457,-100.0,-50.000000,0.0,-5.555556,-19.359508
2,2020-08-03,1,2,0,58,2717,inf,100.000000,0.0,-57.352941,-39.039713
3,2020-08-04,0,2,0,61,6842,-100.0,0.000000,0.0,5.172414,151.821862
4,2020-08-05,0,3,0,55,4630,0.0,50.000000,0.0,-9.836066,-32.329728
...,...,...,...,...,...,...,...,...,...,...,...
148,2020-12-27,0,1,0,290,5188,-100.0,-87.500000,0.0,-19.667590,-70.060018
149,2020-12-28,0,2,0,126,4146,0.0,100.000000,0.0,-56.551724,-20.084811
150,2020-12-29,1,10,0,175,5448,inf,400.000000,0.0,38.888889,31.403763
151,2020-12-30,2,13,0,385,7247,100.0,30.000000,0.0,120.000000,33.021292


In [84]:
DATES = open("/content/gdrive/MyDrive/Colab/COMS6998EMODS/dates.txt").readlines()
DATES = [date.split("\n")[0] for date in DATES]

MONTH = {'08':'Aug_2020', '09':'Sep_2020', '10':"Oct_2020", '11':'Nov_2020', '12':'Dec_2020'}

HOME_DIR = '/content/gdrive/MyDrive/Colab/COMS6998EMODS/Hyderated_Tweets_Hate_Indicator'

Hate_sppech_num = []
for date in tqdm(DATES):
  month = MONTH[date.split("-")[1]]
  csv_path = os.path.join(HOME_DIR, month, date+'.csv')
  daily = pd.read_csv(csv_path, engine='python')
  daily = daily[daily['score'] >= 0.1]
  row_num = daily.shape[0]
  Hate_sppech_num.append(row_num)

hate_df = pd.DataFrame(list(zip(DATES, Hate_sppech_num)), columns =['date', 'count'])
hate_df['Rate']=hate_df['count'].pct_change() * 100
hate_df.to_csv('/content/gdrive/MyDrive/Colab/COMS6998EMODS/Hate_Cnt_US.csv', index=False)

  0%|          | 0/153 [00:00<?, ?it/s]

In [127]:

DATES = open("/content/gdrive/MyDrive/Colab/COMS6998EMODS/dates.txt").readlines()
DATES = [date.split("\n")[0] for date in DATES]

trumps = pd.read_csv("/content/gdrive/MyDrive/Colab/COMS6998EMODS/trumps_with_Hate_indicators.csv")
trumps = trumps.drop(trumps.columns[0], axis=1)
trumps['date']= pd.to_datetime(trumps['date'])
trumps["date"] = trumps["date"].dt.strftime("%Y-%m-%d")
mask = (trumps['date'] >= '2020-08-01') & (trumps['date'] <= '2020-12-31')
trumps = trumps.loc[mask]
trumps = trumps.sort_values(['date'])
trumps = trumps.reset_index(drop=True)

print("The total number tweet of trumps within 153 days: ", trumps.shape[0])

neg_avg, pos_avg, neu_avg = [], [], []

for date in tqdm(DATES):
  daily = trumps.loc[trumps['date'] == date]
  neg_avg.append(daily['NEG'].mean())
  pos_avg.append(daily['POS'].mean())
  neu_avg.append(daily['NEU'].mean())


trumps_score = pd.DataFrame(list(zip(DATES, neg_avg, pos_avg, neu_avg)), columns =['date', 'NEG', 'POS', 'NEU'])

trumps_score['NegRate'] = trumps_score['NEG'].abs().pct_change()
trumps_score['PosRate'] = trumps_score['POS'].abs().pct_change()
trumps_score['NeuRate'] = trumps_score['NEU'].abs().pct_change()

new_df.replace(np.nan, 0, inplace=True)

trumps_score['NegRate'] = trumps_score['NegRate'] * 100
trumps_score['PosRate'] = trumps_score['PosRate'] * 100
trumps_score['NeuRate'] = trumps_score['NeuRate'] * 100

trumps_score.to_csv("/content/gdrive/MyDrive/Colab/COMS6998EMODS/trump_score.csv", index=False)
trumps_score

The total number tweet of trumps within 153 days:  5454


  0%|          | 0/153 [00:00<?, ?it/s]

,date,NEG,POS,NEU,NegRate,PosRate,NeuRate
0,2020-08-01,0.333235,0.301765,0.364824,NaN,NaN,NaN
1,2020-08-02,0.338731,0.296487,0.364718,1.649128,-1.748888,-0.028940
2,2020-08-03,0.347074,0.293630,0.359222,2.463108,-0.963802,-1.506843
3,2020-08-04,0.168625,0.353500,0.477750,-51.415271,20.389758,32.995670
4,2020-08-05,0.343136,0.301455,0.355273,103.490801,-14.722901,-25.636268
...,...,...,...,...,...,...,...
148,2020-12-27,0.011000,0.530444,0.458667,-97.871073,136.197218,77.222965
149,2020-12-28,0.313333,0.185000,0.501833,2748.484848,-65.123586,9.411337
150,2020-12-29,0.575333,0.168000,0.256667,83.617021,-9.189189,-48.854201
151,2020-12-30,0.564375,0.095583,0.339917,-1.904693,-43.105159,32.435065
